In [1]:
import pandas as pd
import numpy as np
import s3fs
import boto3
import io
from census import Census
from us import states
import re

In [2]:
fs = s3fs.S3FileSystem(profile='AdministratorAccess-769392325318')  # or None for default profile

parquet_files = fs.glob("mortgage-data-refined/mke_data/*.parquet")

dfs = [pd.read_parquet(f"s3://{file}", engine='pyarrow', filesystem=fs) for file in parquet_files]

df = pd.concat(dfs, ignore_index=False)
    

In [4]:
def send_dataframe_to_s3_csv(df, bucket, key, profile=None):
    # Set up AWS session
    session = boto3.Session(profile_name=profile) if profile else boto3.Session()
    s3 = session.client('s3')

    # Create in-memory CSV buffer
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)

    # Upload to S3
    s3.put_object(Bucket=bucket, Key=key, Body=csv_buffer.getvalue())
    print(f"✅ Uploaded to s3://{bucket}/{key}")


In [5]:
# Start by uploading this dataframe to S3
file_key = "mke_data/combined_hmda_2018_to_2023.csv"
bucket = 'mortgage-data-refined'
send_dataframe_to_s3_csv(df, bucket, file_key, profile='AdministratorAccess-769392325318')

✅ Uploaded to s3://mortgage-data-refined/mke_data/combined_hmda_2018_to_2023.csv


In [17]:
df.county_code.dtypes

dtype('float64')

In [5]:
df.shape

(139293, 100)

In [33]:
df.loan_purpose.unique().tolist()

[1, 32, 2, 4, 31, 5]

In [47]:
df.hoepa_status.value_counts()

hoepa_status
2    65904
3    57228
1      198
Name: count, dtype: int64

In [46]:
df.hoepa_status.isnull().sum()

0

In [118]:
cols_to_drop = ['rate_spread', 'hoepa_status', 'open_end_line_of_credit', 'interest_rate', 'rate_spread', 
               'hoepa_status', 'total_loan_costs', 'total_points_and_fees', 'origination_charges', 
               'discount_points', 'lender_credits', 'loan_term', 'prepayment_penalty_term', 
               'intro_rate_period', 'negative_amortization', 'interest_only_payment', 
               'balloon_payment', 'other_nonamortizing_features']

In [92]:
new_altered_columns_created = ['purchaser_type', 'loan_type', 'lien_status_category', 'loan_to_value_ratio', 
                              'combined_loan_to_value_ratio']

In [42]:
# Handle duplicates
df.drop_duplicates(inplace=True)

In [14]:
# dropping non WI or nan rows - do this for the Milwaukee set only
# df = df[~((df['state_code'] == 'CA') | (df['state_code'] == 'NV'))]
df.loc[df['state_code'] == 'nan', 'state_code'] = np.nan

In [6]:
# Cleaning instance of incorrect county_code - MKE set only
# df.loc[df['county_code'] == 55075.0, 'county_code'] = 55079.0

In [21]:
# converting county code to int
df['county_code'] = df['county_code'].astype('Int64')

In [10]:
# Dropping the one bad census_tract row - only for Milwaukee
# df = df[~(df['census_tract'] == 55079)]

In [22]:
# Cleaning conforming_loan_limit
df.loc[df['conforming_loan_limit'] == 'C', 'conforming_loan_limit'] = 1
df.loc[df['conforming_loan_limit'] == 'NC', 'conforming_loan_limit'] = 0
df['conforming_loan_limit'] = pd.to_numeric(df['conforming_loan_limit'], errors='coerce').astype('Int64')

In [23]:
# Alter action_taken slightly to group into origination, denial, or fallout and drop those that we don't care about
# df = df[df['action_taken'].isin([1, 2, 3, 4, 5])] # Mke only
df.loc[df['action_taken'] == 1, 'action_taken_category'] = 'Origination'
df.loc[df['action_taken'] == 3, 'action_taken_category'] = 'Denied'
df.loc[df['action_taken'] == 4, 'action_taken_category'] = 'Fallout'
df.loc[df['action_taken'] == 5, 'action_taken_category'] = 'Fallout'
df.loc[df['action_taken'] == 2, 'action_taken_category'] = 'Fallout'
df.loc[df['action_taken'] == 6, 'action_taken_category'] = 'Purchased - Secondary Market'
df.loc[df['action_taken'] == 7, 'action_taken_category'] = 'Preapproval - Denial'
df.loc[df['action_taken'] == 8, 'action_taken_category'] = 'Preapproval - Approved not accepted'

In [25]:
# Create categories for purchaser_type
def categorize_purchaser(code):
    if code in [1, 3]:
        return "GSE"
    elif code == 2:
        return "Government"
    elif code == 0:
        return "Portfolio"
    elif code in [5, 7]:
        return "Private"
    elif code == 6:
        return "Depository"
    elif code == 8:
        return "Affiliate"
    else:
        return "Other"

df["purchaser_category"] = df["purchaser_type"].apply(categorize_purchaser)

In [29]:
# Modifying preapproval column
df.loc[df['preapproval'] == 2, 'preapproval'] = 0

In [30]:
# Modifying loan_type
def categorize_loan_type(code):
    if code == 1:
        return "Conventional"
    elif code == 2:
        return "FHA"
    elif code == 3:
        return "VA"
    elif code == 4:
        return "USDA"
    else:
        return "Other"

df["loan_type_category"] = df["loan_type"].apply(categorize_loan_type)

In [34]:
# only keeping loans which are purchased
# df = df[df['loan_purpose'] == 1] - model only
def categorize_loan_purpose(code): 
    if code == 1: 
        return 'Home Purchase'
    elif code == 2: 
        return 'Home Improvement'
    elif code in [31, 32]: 
        return 'Refinancing'
    elif code == 5: 
        return np.nan
    else: 
        return 'Other'

df['loan_purpose_category'] = df['loan_purpose'].apply(categorize_loan_purpose)

In [35]:
# Categorizing lien_status
def categorize_lien_status(code): 
    if code == 1: 
        return "First Lien"
    elif code == 2: 
        return "Subordinate Lien"
    else: 
        return "Other"

df['lien_status_category'] = df['lien_status'].apply(categorize_lien_status)

In [36]:
# Handling reverse mortgages
# df = df[~(df['reverse_mortgage'] == 1)] - Model only
df.loc[df['reverse_mortgage'] == 1, 'reverse_mortgage_category'] = 'Reverse Mortgage'
df.loc[df['reverse_mortgage'] == 2, 'reverse_mortgage_category'] = 'Not Reverse Mortgage'
df.loc[df['reverse_mortgage'] == 1111, 'reverse_mortgage_category'] = 'Exempt'

df.loc[df['reverse_mortgage'] == 1, 'reverse_mortgage'] = 1
df.loc[df['reverse_mortgage'] == 2, 'reverse_mortgage'] = 0
df.loc[df['reverse_mortgage'] == 1111, 'reverse_mortgage'] = np.nan

In [37]:
# Handling open_end_line_of_credit
# df = df[~(df['open_end_line_of_credit'] == 1)] - Model only
df.loc[df['open_end_line_of_credit'] == 1, 'open_end_line_of_credit_category'] = 'Open Line of Credit'
df.loc[df['open_end_line_of_credit'] == 2, 'open_end_line_of_credit_category'] = 'Not Open Line of Credit'
df.loc[df['open_end_line_of_credit'] == 1111, 'open_end_line_of_credit_category'] = 'Exempt'

df.loc[df['open_end_line_of_credit'] == 1, 'open_end_line_of_credit'] = 1
df.loc[df['open_end_line_of_credit'] == 2, 'open_end_line_of_credit'] = 0
df.loc[df['open_end_line_of_credit'] == 1111, 'open_end_line_of_credit'] = np.nan

In [38]:
# Handling business_or_commercial_purpose
# df = df[~(df['business_or_commercial_purpose'] == 1)] - Model Only
df.loc[df['business_or_commercial_purpose'] == 1, 'business_or_commercial_purpose_category'] = 'Business Purpose'
df.loc[df['business_or_commercial_purpose'] == 2, 'business_or_commercial_purpose_category'] = 'Not Business Purpose'
df.loc[df['business_or_commercial_purpose'] == 1111, 'business_or_commercial_purpose_category'] = 'Exempt'

df.loc[df['business_or_commercial_purpose'] == 1, 'business_or_commercial_purpose'] = 1
df.loc[df['business_or_commercial_purpose'] == 2, 'business_or_commercial_purpose'] = 0
df.loc[df['business_or_commercial_purpose'] == 1111, 'business_or_commercial_purpose'] = np.nan

In [43]:
# Handle the loan_to_value_ratio and combined_loan_to_value_ratio columns
# Also create an effective ltv column
df = df.reset_index(drop=True)
df['combined_loan_to_value_ratio'] = pd.to_numeric(df['combined_loan_to_value_ratio'], errors='coerce')
df['effective_ltv'] = df['loan_to_value_ratio']
df.loc[df['effective_ltv'].isna(), 'effective_ltv'] = df['combined_loan_to_value_ratio']

In [44]:
# Fix the datatype for interest_rate
df['interest_rate'] = pd.to_numeric(df['interest_rate'], errors='coerce')

In [45]:
# rate_spread column
df['rate_spread'] = pd.to_numeric(df['rate_spread'], errors='coerce')

In [49]:
# hoepa_status
df.loc[df['hoepa_status'] == 1, 'hopea_status_category'] = 'High Cost Mortgage'
df.loc[df['hoepa_status'] == 2, 'hopea_status_category'] = 'Not a High Cost Mortgage'
df.loc[df['hoepa_status'] == 3, 'hopea_status_category'] = np.nan

df.loc[df['hoepa_status'] == 1, 'hopea_status'] = 1
df.loc[df['hoepa_status'] == 2, 'hopea_status'] = 0
df.loc[df['hoepa_status'] == 3, 'hopea_status'] = np.nan

In [54]:
# total_loan_costs
df.loc[df['total_loan_costs'].isin(['nan', 'Exempt']), 'total_loan_costs'] = np.nan
df['total_loan_costs'] = pd.to_numeric(df['total_loan_costs'], errors='coerce')

In [58]:
# total_points_and_fees
df.loc[df['total_points_and_fees'].isin(['nan', 'Exempt']), 'total_points_and_fees'] = np.nan
df['total_points_and_fees'] = pd.to_numeric(df['total_points_and_fees'], errors='coerce')

In [62]:
# origination_charges
df.loc[df['origination_charges'].isin(['nan', 'Exempt']), 'origination_charges'] = np.nan
df['origination_charges'] = pd.to_numeric(df['origination_charges'], errors='coerce')

In [66]:
# discount_points
df.loc[df['discount_points'].isin(['nan', 'Exempt']), 'discount_points'] = np.nan
df['discount_points'] = pd.to_numeric(df['discount_points'], errors='coerce')

In [70]:
# lender_credits
df.loc[df['lender_credits'].isin(['nan', 'Exempt']), 'lender_credits'] = np.nan
df['lender_credits'] = pd.to_numeric(df['lender_credits'], errors='coerce')

In [75]:
# loan_term
df.loc[df['loan_term'].isin(['nan', 'Exempt']), 'loan_term'] = np.nan
df['loan_term'] = pd.to_numeric(df['loan_term'], errors='coerce').astype('Int64')

In [79]:
# prepayment_penalty_term
df.loc[df['prepayment_penalty_term'].isin(['nan', 'Exempt']), 'prepayment_penalty_term'] = np.nan
df['prepayment_penalty_term'] = pd.to_numeric(df['prepayment_penalty_term'], errors='coerce').astype('Int64')

In [83]:
# intro_rate_period
df.loc[df['intro_rate_period'].isin(['nan', 'Exempt']), 'intro_rate_period'] = np.nan
df['intro_rate_period'] = pd.to_numeric(df['intro_rate_period'], errors='coerce').astype('Int64')

In [87]:
# negative_amortization
df.loc[df['negative_amortization'] == 1, 'negative_amortization_category'] = 'Negative Amortization Features'
df.loc[df['negative_amortization'] == 2, 'negative_amortization_category'] = 'No Negative Amortization Features'
df.loc[df['negative_amortization'] == 1111, 'negative_amortization_category'] = np.nan

df.loc[df['negative_amortization'] == 1, 'negative_amortization'] = 1
df.loc[df['negative_amortization'] == 2, 'negative_amortization'] = 0
df.loc[df['negative_amortization'] == 1111, 'negative_amortization'] = np.nan

In [91]:
# interest_only_payment
df.loc[df['interest_only_payment'] == 1, 'interest_only_payment_category'] = 'Interest-Only Payments'
df.loc[df['interest_only_payment'] == 2, 'interest_only_payment_category'] = 'No Interest-Only Payments'
df.loc[df['interest_only_payment'] == 1111, 'interest_only_payment_category'] = np.nan

df.loc[df['interest_only_payment'] == 1, 'interest_only_payment'] = 1
df.loc[df['interest_only_payment'] == 2, 'interest_only_payment'] = 0
df.loc[df['interest_only_payment'] == 1111, 'interest_only_payment'] = np.nan

In [95]:
# interest_only_payment
df.loc[df['balloon_payment'] == 1, 'balloon_payment_category'] = 'Balloon Payment'
df.loc[df['balloon_payment'] == 2, 'balloon_payment_category'] = 'No Balloon Payment'
df.loc[df['balloon_payment'] == 1111, 'balloon_payment_category'] = np.nan

df.loc[df['balloon_payment'] == 1, 'balloon_payment'] = 1
df.loc[df['balloon_payment'] == 2, 'balloon_payment'] = 0
df.loc[df['balloon_payment'] == 1111, 'balloon_payment'] = np.nan

In [99]:
# other_nonamortizing_features
df.loc[df['other_nonamortizing_features'] == 1, 'other_nonamortizing_features_category'] = 'Other non-fully amortizing features'
df.loc[df['other_nonamortizing_features'] == 2, 'other_nonamortizing_features_category'] = 'No other non-fully amortizing features'
df.loc[df['other_nonamortizing_features'] == 1111, 'other_nonamortizing_features_category'] = np.nan

df.loc[df['other_nonamortizing_features'] == 1, 'other_nonamortizing_features'] = 1
df.loc[df['other_nonamortizing_features'] == 2, 'other_nonamortizing_features'] = 0
df.loc[df['other_nonamortizing_features'] == 1111, 'other_nonamortizing_features'] = np.nan

In [106]:
# property_value
df.loc[df['property_value'].isin(['nan', 'Exempt']), 'property_value'] = np.nan
df['property_value'] = pd.to_numeric(df['property_value'], errors='coerce').astype('Int64')

In [119]:
# construction_method
df.loc[df['construction_method'] == 1, 'construction_method_category'] = 'Site-built'
df.loc[df['construction_method'] == 2, 'construction_method_category'] = 'Manufactured Home'

In [120]:
# occupancy_type
df.loc[df['occupancy_type'] == 1, 'occupancy_type_category'] = 'Principal Residence'
df.loc[df['occupancy_type'] == 2, 'occupancy_type_category'] = 'Second Residence'
df.loc[df['occupancy_type'] == 3, 'occupancy_type_category'] = 'Investment Property'

In [124]:
# manufactured_home_secured_property_type
df.loc[df['manufactured_home_secured_property_type'] == 1, 'manufactured_home_secured_property_type_category'] = 'Manufactured home and land'
df.loc[df['manufactured_home_secured_property_type'] == 2, 'manufactured_home_secured_property_type_category'] = 'Manufactured Home - Not Land'
df.loc[df['manufactured_home_secured_property_type'] == 3, 'manufactured_home_secured_property_type_category'] = np.nan
df.loc[df['manufactured_home_secured_property_type'] == 1111, 'manufactured_home_secured_property_type_category'] = np.nan

df.loc[df['manufactured_home_secured_property_type'] == 3, 'manufactured_home_secured_property_type'] = np.nan
df.loc[df['manufactured_home_secured_property_type'] == 1111, 'manufactured_home_secured_property_type'] = np.nan

In [128]:
# manufactured_home_land_property_interest
df.loc[df['manufactured_home_land_property_interest'] == 1, 'manufactured_home_land_property_interest_category'] = 'Direct Ownership'
df.loc[df['manufactured_home_land_property_interest'] == 2, 'manufactured_home_land_property_interest_category'] = 'Indirect Ownership'
df.loc[df['manufactured_home_land_property_interest'] == 3, 'manufactured_home_land_property_interest_category'] = 'Paid Leasehold'
df.loc[df['manufactured_home_land_property_interest'] == 4, 'manufactured_home_land_property_interest_category'] = 'Unpaid Leasehold'
df.loc[df['manufactured_home_land_property_interest'] == 5, 'manufactured_home_land_property_interest_category'] = np.nan
df.loc[df['manufactured_home_land_property_interest'] == 1111, 'manufactured_home_land_property_interest_category'] = np.nan

df.loc[df['manufactured_home_land_property_interest'] == 5, 'manufactured_home_land_property_interest'] = np.nan
df.loc[df['manufactured_home_land_property_interest'] == 1111, 'manufactured_home_land_property_interest'] = np.nan

In [147]:
# Cleaning up total_units
df.loc[df['total_units'].isin([1, 1.0, '1']), 'total_units'] = 1
df.loc[df['total_units'].isin([2, 2.0, '2']), 'total_units'] = 2
df.loc[df['total_units'].isin([3, 4.0, '3']), 'total_units'] = 3
df.loc[df['total_units'].isin([4, 4.0, '4']), 'total_units'] = 4

# Create bucketed column
def total_units_bucketed(code): 
    if pd.isna(code): 
        return "Unknown"
    try: 
        val = int(code)
        return str(val) if val <= 4 else "5+"
    except: 
        return "5+"

df['total_units_bucketed'] = df['total_units'].apply(total_units_bucketed)

In [154]:
# multifamily_affordable_units
df["multifamily_affordable_units"] = pd.to_numeric(df["multifamily_affordable_units"], errors="coerce").astype('Int64')
df["multifamily_affordable_units"] = df["multifamily_affordable_units"].fillna(0)

In [161]:
# income
df['income'] = pd.to_numeric(df['income'], errors='coerce').astype('Int64')
df['income'] = df['income'] * 1000

In [169]:
# debt_to_income_ratio
def transform_dti(x): 
    x = str(x)
    if x in ['nan', 'Exempt']: 
        return np.nan
    try: 
        val = int(x)
        if val < 20: 
            return '<20%'
        elif val >= 20 and val < 30: 
            return '20%-<30%'
        elif val >= 30 and val < 36: 
            return '30%-<36%'
        elif val >= 36 and val < 40: 
            return '36%-<40%'
        elif val >= 40 and val < 46: 
            return '40%-<46%'
        elif val >= 46 and val < 50: 
            return '46%-<50%'
        elif val >= 50 and val <= 60: 
            return '50%-60%'
        elif val > 60: 
            return '>60%'
        else: 
            return x
    except: 
        return x 

df['debt_to_income_ratio'] = df['debt_to_income_ratio'].apply(transform_dti)

In [175]:
# applicant_credit_score_type
df.loc[df['applicant_credit_score_type'] == 1, 'applicant_credit_score_type_category'] = 'Equifax Beacon 5.0'
df.loc[df['applicant_credit_score_type'] == 2, 'applicant_credit_score_type_category'] = 'Experian Fair Isaac'
df.loc[df['applicant_credit_score_type'] == 3, 'applicant_credit_score_type_category'] = 'FICO Risk Score Classic 04'
df.loc[df['applicant_credit_score_type'] == 4, 'applicant_credit_score_type_category'] = 'FICO Risk Score Classic 98'
df.loc[df['applicant_credit_score_type'] == 5, 'applicant_credit_score_type_category'] = 'VantageScore 2.0'
df.loc[df['applicant_credit_score_type'] == 6, 'applicant_credit_score_type_category'] = 'VantageScore 3.0'
df.loc[df['applicant_credit_score_type'] == 7, 'applicant_credit_score_type_category'] = 'More than one credit score model'
df.loc[df['applicant_credit_score_type'] == 8, 'applicant_credit_score_type_category'] = 'Other credit scoring model'
df.loc[df['applicant_credit_score_type'] == 9, 'applicant_credit_score_type_category'] = np.nan
df.loc[df['applicant_credit_score_type'] == 1111, 'applicant_credit_score_type_category'] = np.nan

df.loc[df['applicant_credit_score_type'] == 9, 'applicant_credit_score_type'] = np.nan
df.loc[df['applicant_credit_score_type'] == 1111, 'applicant_credit_score_type'] = np.nan

In [177]:
# co_applicant_credit_score_type
df.loc[df['co_applicant_credit_score_type'] == 1, 'co_applicant_credit_score_type_category'] = 'Equifax Beacon 5.0'
df.loc[df['co_applicant_credit_score_type'] == 2, 'co_applicant_credit_score_type_category'] = 'Experian Fair Isaac'
df.loc[df['co_applicant_credit_score_type'] == 3, 'co_applicant_credit_score_type_category'] = 'FICO Risk Score Classic 04'
df.loc[df['co_applicant_credit_score_type'] == 4, 'co_applicant_credit_score_type_category'] = 'FICO Risk Score Classic 98'
df.loc[df['co_applicant_credit_score_type'] == 5, 'co_applicant_credit_score_type_category'] = 'VantageScore 2.0'
df.loc[df['co_applicant_credit_score_type'] == 6, 'co_applicant_credit_score_type_category'] = 'VantageScore 3.0'
df.loc[df['co_applicant_credit_score_type'] == 7, 'co_applicant_credit_score_type_category'] = 'More than one credit score model'
df.loc[df['co_applicant_credit_score_type'] == 8, 'co_applicant_credit_score_type_category'] = 'Other credit scoring model'
df.loc[df['co_applicant_credit_score_type'] == 9, 'co_applicant_credit_score_type_category'] = 'No Co-Applicant'
df.loc[df['co_applicant_credit_score_type'] == 10, 'co_applicant_credit_score_type_category'] = np.nan
df.loc[df['co_applicant_credit_score_type'] == 1111, 'co_applicant_credit_score_type_category'] = np.nan

df.loc[df['co_applicant_credit_score_type'] == 9, 'co_applicant_credit_score_type'] = np.nan
df.loc[df['co_applicant_credit_score_type'] == 1111, 'co_applicant_credit_score_type'] = np.nan

In [195]:
# applicant_ethnicity_X
for x in range(1, 6): 
    df.loc[df[f"applicant_ethnicity_{x}"] == 1, f"applicant_ethnicity_{x}_category"] = 'Hispanic or Latino'
    df.loc[df[f"applicant_ethnicity_{x}"] == 2, f"applicant_ethnicity_{x}_category"] = 'Not Hispanic or Latino'
    df.loc[df[f"applicant_ethnicity_{x}"] == 3, f"applicant_ethnicity_{x}_category"] = np.nan
    df.loc[df[f"applicant_ethnicity_{x}"] == 4, f"applicant_ethnicity_{x}_category"] = np.nan
    df.loc[df[f"applicant_ethnicity_{x}"] == 11, f"applicant_ethnicity_{x}_category"] = 'Mexican'
    df.loc[df[f"applicant_ethnicity_{x}"] == 12, f"applicant_ethnicity_{x}_category"] = 'Puerto Rican'
    df.loc[df[f"applicant_ethnicity_{x}"] == 13, f"applicant_ethnicity_{x}_category"] = 'Cuban'
    df.loc[df[f"applicant_ethnicity_{x}"] == 14, f"applicant_ethnicity_{x}_category"] = 'Other Hispanic or Latino'
    
    df.loc[df[f"applicant_ethnicity_{x}"] == 3, f"applicant_ethnicity_{x}"] = np.nan
    df.loc[df[f"applicant_ethnicity_{x}"] == 4, f"applicant_ethnicity_{x}"] = np.nan
    df[f"applicant_ethnicity_{x}"] = pd.to_numeric(df[f"applicant_ethnicity_{x}"], errors="coerce").astype('Int64')


In [196]:
# co_applicant_ethnicity_X
for x in range(1, 6): 
    df.loc[df[f"co_applicant_ethnicity_{x}"] == 1, f"co_applicant_ethnicity_{x}_category"] = 'Hispanic or Latino'
    df.loc[df[f"co_applicant_ethnicity_{x}"] == 2, f"co_applicant_ethnicity_{x}_category"] = 'Not Hispanic or Latino'
    df.loc[df[f"co_applicant_ethnicity_{x}"] == 3, f"co_applicant_ethnicity_{x}_category"] = np.nan
    df.loc[df[f"co_applicant_ethnicity_{x}"] == 4, f"co_applicant_ethnicity_{x}_category"] = np.nan
    df.loc[df[f"co_applicant_ethnicity_{x}"] == 5, f"co_applicant_ethnicity_{x}_category"] = np.nan
    df.loc[df[f"co_applicant_ethnicity_{x}"] == 11, f"co_applicant_ethnicity_{x}_category"] = 'Mexican'
    df.loc[df[f"co_applicant_ethnicity_{x}"] == 12, f"co_applicant_ethnicity_{x}_category"] = 'Puerto Rican'
    df.loc[df[f"co_applicant_ethnicity_{x}"] == 13, f"co_applicant_ethnicity_{x}_category"] = 'Cuban'
    df.loc[df[f"co_applicant_ethnicity_{x}"] == 14, f"co_applicant_ethnicity_{x}_category"] = 'Other Hispanic or Latino'
    
    df.loc[df[f"co_applicant_ethnicity_{x}"] == 3, f"co_applicant_ethnicity_{x}"] = np.nan
    df.loc[df[f"co_applicant_ethnicity_{x}"] == 4, f"co_applicant_ethnicity_{x}"] = np.nan
    df[f"co_applicant_ethnicity_{x}"] = pd.to_numeric(df[f"co_applicant_ethnicity_{x}"], errors="coerce").astype('Int64')


In [198]:
# create no co applicant column
df['co_applicant_present'] = (df['co_applicant_ethnicity_1'] != 5).astype('Int64')

In [206]:
# applicant_ethnicity_observed
df.loc[df['applicant_ethnicity_observed'] == 1, 'applicant_ethnicity_observed_category'] = 'Ethnicity Collected on Basis of Observation'
df.loc[df['applicant_ethnicity_observed'] == 2, 'applicant_ethnicity_observed_category'] = 'Ethnicity Not Based on Observation'
df.loc[df['applicant_ethnicity_observed'] == 3, 'applicant_ethnicity_observed_category'] = np.nan

df.loc[df['applicant_ethnicity_observed'] == 1, 'applicant_enthnicity_observed'] = 1
df.loc[df['applicant_ethnicity_observed'] == 2, 'applicant_enthnicity_observed'] = 0
df.loc[df['applicant_ethnicity_observed'] == 3, 'applicant_enthnicity_observed'] = np.nan

In [207]:
# co applicant_ethnicity_observed
df.loc[df['co_applicant_ethnicity_observed'] == 1, 'co_applicant_ethnicity_observed_category'] = 'Ethnicity Collected on Basis of Observation'
df.loc[df['co_applicant_ethnicity_observed'] == 2, 'co_applicant_ethnicity_observed_category'] = 'Ethnicity Not Based on Observation'
df.loc[df['co_applicant_ethnicity_observed'] == 3, 'co_applicant_ethnicity_observed_category'] = np.nan
df.loc[df['co_applicant_ethnicity_observed'] == 4, 'co_applicant_ethnicity_observed_category'] = np.nan

df.loc[df['co_applicant_ethnicity_observed'] == 1, 'co_applicant_enthnicity_observed'] = 1
df.loc[df['co_applicant_ethnicity_observed'] == 2, 'co_applicant_enthnicity_observed'] = 0
df.loc[df['co_applicant_ethnicity_observed'] == 3, 'co_applicant_enthnicity_observed'] = np.nan
df.loc[df['co_applicant_ethnicity_observed'] == 4, 'co_applicant_enthnicity_observed'] = np.nan

In [211]:
# applicant_race_X
for x in range(1, 6): 
    df[f"applicant_race_{x}"] = pd.to_numeric(df[f"applicant_race_{x}"], errors="coerce").astype('Int64')
    df.loc[df[f"applicant_race_{x}"] == 1, f"applicant_race_{x}_category"] = 'American Indian or Alaska Native'
    df.loc[df[f"applicant_race_{x}"] == 2, f"applicant_race_{x}_category"] = 'Asian'
    df.loc[df[f"applicant_race_{x}"] == 21, f"applicant_race_{x}_category"] = 'Asian Indian'
    df.loc[df[f"applicant_race_{x}"] == 22, f"applicant_race_{x}_category"] = 'Chinese'
    df.loc[df[f"applicant_race_{x}"] == 23, f"applicant_race_{x}_category"] = 'Filipino'
    df.loc[df[f"applicant_race_{x}"] == 24, f"applicant_race_{x}_category"] = 'Japanese'
    df.loc[df[f"applicant_race_{x}"] == 25, f"applicant_race_{x}_category"] = 'Korean'
    df.loc[df[f"applicant_race_{x}"] == 26, f"applicant_race_{x}_category"] = 'Vietnamese'
    df.loc[df[f"applicant_race_{x}"] == 27, f"applicant_race_{x}_category"] = 'Other Asian'
    df.loc[df[f"applicant_race_{x}"] == 3, f"applicant_race_{x}_category"] = 'Black or African American'
    df.loc[df[f"applicant_race_{x}"] == 4, f"applicant_race_{x}_category"] = 'Native Hawaiian or Other Pacific Islander'
    df.loc[df[f"applicant_race_{x}"] == 41, f"applicant_race_{x}_category"] = 'Native Hawaiian'
    df.loc[df[f"applicant_race_{x}"] == 42, f"applicant_race_{x}_category"] = 'Guamanian or Chamorro'
    df.loc[df[f"applicant_race_{x}"] == 43, f"applicant_race_{x}_category"] = 'Samoan'
    df.loc[df[f"applicant_race_{x}"] == 44, f"applicant_race_{x}_category"] = 'Other Pacific Islander'
    df.loc[df[f"applicant_race_{x}"] == 5, f"applicant_race_{x}_category"] = 'White'
    df.loc[df[f"applicant_race_{x}"] == 6, f"applicant_race_{x}_category"] = np.nan
    df.loc[df[f"applicant_race_{x}"] == 7, f"applicant_race_{x}_category"] = np.nan
    
    df.loc[df[f"applicant_race_{x}"] == 1, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 2, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 21, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 22, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 23, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 24, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 25, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 26, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 27, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 3, f"applicant_race_{x}_category_grouped"] = 'Black'
    df.loc[df[f"applicant_race_{x}"] == 4, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 41, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 42, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 43, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 44, f"applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"applicant_race_{x}"] == 5, f"applicant_race_{x}_category_grouped"] = 'White'
    df.loc[df[f"applicant_race_{x}"] == 6, f"applicant_race_{x}_category_grouped"] = np.nan
    df.loc[df[f"applicant_race_{x}"] == 7, f"applicant_race_{x}_category_grouped"] = np.nan
    df.loc[df[f"applicant_race_{x}"] == 8, f"applicant_race_{x}_category_grouped"] = np.nan
    
    df.loc[df[f"applicant_race_{x}"] == 6, f"applicant_race_{x}"] = np.nan
    df.loc[df[f"applicant_race_{x}"] == 7, f"applicant_race_{x}"] = np.nan
    df.loc[df[f"applicant_race_{x}"] == 8, f"applicant_race_{x}"] = np.nan

In [213]:
# co_applicant_race_X
for x in range(1, 6): 
    df[f"co_applicant_race_{x}"] = pd.to_numeric(df[f"co_applicant_race_{x}"], errors="coerce").astype('Int64')
    df.loc[df[f"co_applicant_race_{x}"] == 1, f"co_applicant_race_{x}_category"] = 'American Indian or Alaska Native'
    df.loc[df[f"co_applicant_race_{x}"] == 2, f"co_applicant_race_{x}_category"] = 'Asian'
    df.loc[df[f"co_applicant_race_{x}"] == 21, f"co_applicant_race_{x}_category"] = 'Asian Indian'
    df.loc[df[f"co_applicant_race_{x}"] == 22, f"co_applicant_race_{x}_category"] = 'Chinese'
    df.loc[df[f"co_applicant_race_{x}"] == 23, f"co_applicant_race_{x}_category"] = 'Filipino'
    df.loc[df[f"co_applicant_race_{x}"] == 24, f"co_applicant_race_{x}_category"] = 'Japanese'
    df.loc[df[f"co_applicant_race_{x}"] == 25, f"co_applicant_race_{x}_category"] = 'Korean'
    df.loc[df[f"co_applicant_race_{x}"] == 26, f"co_applicant_race_{x}_category"] = 'Vietnamese'
    df.loc[df[f"co_applicant_race_{x}"] == 27, f"co_applicant_race_{x}_category"] = 'Other Asian'
    df.loc[df[f"co_applicant_race_{x}"] == 3, f"co_applicant_race_{x}_category"] = 'Black or African American'
    df.loc[df[f"co_applicant_race_{x}"] == 4, f"co_applicant_race_{x}_category"] = 'Native Hawaiian or Other Pacific Islander'
    df.loc[df[f"co_applicant_race_{x}"] == 41, f"co_applicant_race_{x}_category"] = 'Native Hawaiian'
    df.loc[df[f"co_applicant_race_{x}"] == 42, f"co_applicant_race_{x}_category"] = 'Guamanian or Chamorro'
    df.loc[df[f"co_applicant_race_{x}"] == 43, f"co_applicant_race_{x}_category"] = 'Samoan'
    df.loc[df[f"co_applicant_race_{x}"] == 44, f"co_applicant_race_{x}_category"] = 'Other Pacific Islander'
    df.loc[df[f"co_applicant_race_{x}"] == 5, f"co_applicant_race_{x}_category"] = 'White'
    df.loc[df[f"co_applicant_race_{x}"] == 6, f"co_applicant_race_{x}_category"] = np.nan
    df.loc[df[f"co_applicant_race_{x}"] == 7, f"co_applicant_race_{x}_category"] = np.nan
    df.loc[df[f"co_applicant_race_{x}"] == 8, f"co_applicant_race_{x}_category"] = np.nan
    
    df.loc[df[f"co_applicant_race_{x}"] == 1, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 2, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 21, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 22, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 23, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 24, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 25, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 26, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 27, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 3, f"co_applicant_race_{x}_category_grouped"] = 'Black'
    df.loc[df[f"co_applicant_race_{x}"] == 4, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 41, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 42, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 43, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 44, f"co_applicant_race_{x}_category_grouped"] = 'AAPI'
    df.loc[df[f"co_applicant_race_{x}"] == 5, f"co_applicant_race_{x}_category_grouped"] = 'White'
    df.loc[df[f"co_applicant_race_{x}"] == 6, f"co_applicant_race_{x}_category_grouped"] = np.nan
    df.loc[df[f"co_applicant_race_{x}"] == 7, f"co_applicant_race_{x}_category_grouped"] = np.nan
    df.loc[df[f"co_applicant_race_{x}"] == 8, f"co_applicant_race_{x}_category_grouped"] = np.nan
    
    df.loc[df[f"co_applicant_race_{x}"] == 6, f"co_applicant_race_{x}"] = np.nan
    df.loc[df[f"co_applicant_race_{x}"] == 7, f"co_applicant_race_{x}"] = np.nan
    df.loc[df[f"co_applicant_race_{x}"] == 8, f"co_applicant_race_{x}"] = np.nan

In [217]:
# applicant_race_observed
df.loc[df['applicant_race_observed'] == 1, 'applicant_race_observed_category'] = 'Collected on basis of visual observation'
df.loc[df['applicant_race_observed'] == 2, 'applicant_race_observed_category'] = 'Not Collected on basis of visual observation'
df.loc[df['applicant_race_observed'] == 3, 'applicant_race_observed_category'] = np.nan

df.loc[df['applicant_race_observed'] == 1, 'applicant_race_observed'] = 1
df.loc[df['applicant_race_observed'] == 2, 'applicant_race_observed'] = 0
df.loc[df['applicant_race_observed'] == 3, 'applicant_race_observed'] = np.nan

In [221]:
# applicant_race_observed
df.loc[df['co_applicant_race_observed'] == 1, 'co_applicant_race_observed_category'] = 'Collected on basis of visual observation'
df.loc[df['co_applicant_race_observed'] == 2, 'co_applicant_race_observed_category'] = 'Not Collected on basis of visual observation'
df.loc[df['co_applicant_race_observed'] == 3, 'co_applicant_race_observed_category'] = np.nan
df.loc[df['co_applicant_race_observed'] == 4, 'co_applicant_race_observed_category'] = np.nan

df.loc[df['co_applicant_race_observed'] == 1, 'co_applicant_race_observed'] = 1
df.loc[df['co_applicant_race_observed'] == 2, 'co_applicant_race_observed'] = 0
df.loc[df['co_applicant_race_observed'] == 3, 'co_applicant_race_observed'] = np.nan
df.loc[df['co_applicant_race_observed'] == 4, 'co_applicant_race_observed'] = np.nan

In [225]:
# applicant_sex
df.loc[df['applicant_sex'] == 1, 'applicant_sex_category'] = 'Male'
df.loc[df['applicant_sex'] == 2, 'applicant_sex_category'] = 'Female'
df.loc[df['applicant_sex'] == 3, 'applicant_sex_category'] = np.nan
df.loc[df['applicant_sex'] == 4, 'applicant_sex_category'] = np.nan
df.loc[df['applicant_sex'] == 6, 'applicant_sex_category'] = 'Male and Female'

df.loc[df['applicant_sex'] == 1, 'applicant_sex'] = 1
df.loc[df['applicant_sex'] == 2, 'applicant_sex'] = 2
df.loc[df['applicant_sex'] == 3, 'applicant_sex'] = np.nan
df.loc[df['applicant_sex'] == 4, 'applicant_sex'] = np.nan
df.loc[df['applicant_sex'] == 6, 'applicant_sex'] = 3

In [229]:
# co_applicant_sex
df.loc[df['co_applicant_sex'] == 1, 'co_applicant_sex_category'] = 'Male'
df.loc[df['co_applicant_sex'] == 2, 'co_applicant_sex_category'] = 'Female'
df.loc[df['co_applicant_sex'] == 3, 'co_applicant_sex_category'] = np.nan
df.loc[df['co_applicant_sex'] == 4, 'co_applicant_sex_category'] = np.nan
df.loc[df['co_applicant_sex'] == 5, 'co_applicant_sex_category'] = np.nan
df.loc[df['co_applicant_sex'] == 6, 'co_applicant_sex_category'] = 'Male and Female'

df.loc[df['co_applicant_sex'] == 1, 'co_applicant_sex'] = 1
df.loc[df['co_applicant_sex'] == 2, 'co_applicant_sex'] = 2
df.loc[df['co_applicant_sex'] == 3, 'co_applicant_sex'] = np.nan
df.loc[df['co_applicant_sex'] == 4, 'co_applicant_sex'] = np.nan
df.loc[df['co_applicant_sex'] == 5, 'co_applicant_sex'] = np.nan
df.loc[df['co_applicant_sex'] == 6, 'co_applicant_sex'] = 3

In [233]:
# applicant_sex_observed
df.loc[df['applicant_sex_observed'] == 1, 'applicant_sex_observed_category'] = 'Collected on basis of visual observation'
df.loc[df['applicant_sex_observed'] == 2, 'applicant_sex_observed_category'] = 'Not Collected on basis of visual observation'
df.loc[df['applicant_sex_observed'] == 3, 'applicant_sex_observed_category'] = np.nan


df.loc[df['applicant_sex_observed'] == 1, 'applicant_sex_observed'] = 1
df.loc[df['applicant_sex_observed'] == 2, 'applicant_sex_observed'] = 0
df.loc[df['applicant_sex_observed'] == 3, 'applicant_sex_observed'] = np.nan

In [234]:
# co_applicant_sex_observed
df.loc[df['co_applicant_sex_observed'] == 1, 'co_applicant_sex_observed_category'] = 'Collected on basis of visual observation'
df.loc[df['co_applicant_sex_observed'] == 2, 'co_applicant_sex_observed_category'] = 'Not Collected on basis of visual observation'
df.loc[df['co_applicant_sex_observed'] == 3, 'co_applicant_sex_observed_category'] = np.nan
df.loc[df['co_applicant_sex_observed'] == 4, 'co_applicant_sex_observed_category'] = np.nan


df.loc[df['co_applicant_sex_observed'] == 1, 'co_applicant_sex_observed'] = 1
df.loc[df['co_applicant_sex_observed'] == 2, 'co_applicant_sex_observed'] = 0
df.loc[df['co_applicant_sex_observed'] == 3, 'co_applicant_sex_observed'] = np.nan
df.loc[df['co_applicant_sex_observed'] == 4, 'co_applicant_sex_observed'] = np.nan

In [245]:
# applicant_age
df.loc[df['applicant_age'].astype('str') == '8888'] = np.nan
df.loc[df['applicant_age'].astype('str') == '8888.0'] = np.nan
    

In [249]:
# co_applicant_age
df.loc[df['co_applicant_age'].astype('str') == '8888'] = np.nan
df.loc[df['co_applicant_age'].astype('str') == '8888.0'] = np.nan
df.loc[df['co_applicant_age'].astype('str') == '9999'] = np.nan
df.loc[df['co_applicant_age'].astype('str') == '9999.0'] = np.nan

In [254]:
# applicant_age_above_62
df.loc[df['applicant_age_above_62'] == 'Yes', 'applicant_age_above_62'] = 1
df.loc[df['applicant_age_above_62'] == 'No', 'applicant_age_above_62'] = 0
df[f"applicant_age_above_62"] = pd.to_numeric(df[f"applicant_age_above_62"], errors="coerce").astype('Int64')

In [261]:
# co_applicant_age_above_62
df.loc[df['co_applicant_age_above_62'] == 'Yes', 'co_applicant_age_above_62'] = 1
df.loc[df['co_applicant_age_above_62'] == 'No', 'co_applicant_age_above_62'] = 0
df[f"coapplicant_age_above_62"] = pd.to_numeric(df[f"co_applicant_age_above_62"], errors="coerce").astype('Int64')

/var/folders/88/q8wq3ss94k3bvmkq1j1f_j0w0000gn/T/ipykernel_91022/1508401795.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"coapplicant_age_above_62"] = pd.to_numeric(df[f"co_applicant_age_above_62"], errors="coerce").astype('Int64')


In [266]:
# submission_of_application
df[f"submission_of_application"] = pd.to_numeric(df[f"submission_of_application"], errors="coerce").astype('Int64')
df.loc[df['submission_of_application'] == 1, 'submission_of_application_category'] = 'Submitted directly to institution'
df.loc[df['submission_of_application'] == 2, 'submission_of_application_category'] = 'Not Submitted to institution'
df.loc[df['submission_of_application'] == 3, 'submission_of_application_category'] = np.nan
df.loc[df['submission_of_application'] == 1111, 'submission_of_application_category'] = 'Exempt'

df.loc[df['submission_of_application'] == 1, 'submission_of_application'] = 1
df.loc[df['submission_of_application'] == 2, 'submission_of_application'] = 0
df.loc[df['submission_of_application'] == 3, 'submission_of_application'] = np.nan
df.loc[df['submission_of_application'] == 11111, 'submission_of_application'] = np.nan

In [271]:
# initially_payable_to_institution
df[f"initially_payable_to_institution"] = pd.to_numeric(df[f"initially_payable_to_institution"], errors="coerce").astype('Int64')
df.loc[df['initially_payable_to_institution'] == 1, 'initially_payable_to_institution_category'] = 'Initially Payable to Institution'
df.loc[df['initially_payable_to_institution'] == 2, 'initially_payable_to_institution_category'] = 'Not Initially Payable to Institution'
df.loc[df['initially_payable_to_institution'] == 3, 'initially_payable_to_institution_category'] = np.nan
df.loc[df['initially_payable_to_institution'] == 1111, 'initially_payable_to_institution_category'] = 'Exempt'

df.loc[df['initially_payable_to_institution'] == 1, 'initially_payable_to_institution'] = 1
df.loc[df['initially_payable_to_institution'] == 2, 'initially_payable_to_institution'] = 0
df.loc[df['initially_payable_to_institution'] == 3, 'initially_payable_to_institution'] = np.nan
df.loc[df['initially_payable_to_institution'] == 1111, 'initially_payable_to_institution'] = np.nan

In [276]:
# aus_X
for x in range(1, 6): 
    df[f"aus_{x}"] = pd.to_numeric(df[f"aus_{x}"], errors='coerce').astype('Int64')
    df.loc[df[f"aus_{x}"] == 1, f"aus_{x}_category"] = 'Desktop Underwriter'
    df.loc[df[f"aus_{x}"] == 2, f"aus_{x}_category"] = 'Loan Prospector or Loan Product Advisor'
    df.loc[df[f"aus_{x}"] == 3, f"aus_{x}_category"] = 'Technology Open to Approved Lenders'
    df.loc[df[f"aus_{x}"] == 4, f"aus_{x}_category"] = 'Guaranteed Underwriting System'
    df.loc[df[f"aus_{x}"] == 5, f"aus_{x}_category"] = 'Other'
    df.loc[df[f"aus_{x}"] == 6, f"aus_{x}_category"] = np.nan
    df.loc[df[f"aus_{x}"] == 7, f"aus_{x}_category"] = 'Internal Proprietary System'
    df.loc[df[f"aus_{x}"] == 1111, f"aus_{x}_category"] = 'Exempt'
    
    df.loc[df[f"aus_{x}"] == 6, f"aus_{x}"] = np.nan
    df.loc[df[f"aus_{x}"] == 1111, f"aus_{x}"] = np.nan
    

In [280]:
# denial_reason_X
for x in range(1, 5): 
    df[f"denial_reason_{x}"] = pd.to_numeric(df[f"denial_reason_{x}"], errors='coerce').astype('Int64')
    df.loc[df[f"denial_reason_{x}"] == 1, f"denial_reason_{x}_category"] = 'Debt to Income Ratio'
    df.loc[df[f"denial_reason_{x}"] == 2, f"denial_reason_{x}_category"] = 'Employment History'
    df.loc[df[f"denial_reason_{x}"] == 3, f"denial_reason_{x}_category"] = 'Credit History'
    df.loc[df[f"denial_reason_{x}"] == 4, f"denial_reason_{x}_category"] = 'Collateral'
    df.loc[df[f"denial_reason_{x}"] == 5, f"denial_reason_{x}_category"] = 'Insufficient cash - downpayment or closing costs'
    df.loc[df[f"denial_reason_{x}"] == 6, f"denial_reason_{x}_category"] = 'Unverifiable Information'
    df.loc[df[f"denial_reason_{x}"] == 7, f"denial_reason_{x}_category"] = 'Credit application incomplete'
    df.loc[df[f"denial_reason_{x}"] == 8, f"denial_reason_{x}_category"] = 'Mortgage insurance denied'
    df.loc[df[f"denial_reason_{x}"] == 9, f"denial_reason_{x}_category"] = 'Other'
    df.loc[df[f"denial_reason_{x}"] == 10, f"denial_reason_{x}_category"] = np.nan
    
    df.loc[df[f"denial_reason_{x}"] == 10, f"denial_reason_{x}"] = np.nan

In [284]:
# tract_population
df['tract_population'] = pd.to_numeric(df['tract_population'], errors='coerce').astype('Int64')

In [291]:
# ffiec_msa_md_median_family_income
df['ffiec_msa_md_median_family_income'] = pd.to_numeric(df['ffiec_msa_md_median_family_income'], errors='coerce').astype('Int64')

In [298]:
# tract_owner_occupied_units
df['tract_owner_occupied_units'] = pd.to_numeric(df['tract_owner_occupied_units'], errors='coerce').astype('Int64')

In [302]:
# tract_one_to_four_family_homes
df['tract_one_to_four_family_homes'] = pd.to_numeric(df['tract_one_to_four_family_homes'], errors='coerce').astype('Int64')

In [306]:
# tract_median_age_of_housing_units
df['tract_median_age_of_housing_units'] = pd.to_numeric(df['tract_median_age_of_housing_units'], errors='coerce').astype('Int64')

In [160]:
df.loc[df['income'] == 'Exempt']

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,hopea_status,negative_amortization_category,interest_only_payment_category,balloon_payment_category,other_nonamortizing_features_category,construction_method_category,occupancy_type_category,manufactured_home_secured_property_type_category,manufactured_home_land_property_interest_category,total_units_bucketed


In [307]:
df.combined_loan_to_value_ratio.dtype

dtype('float64')

In [308]:
df.combined_loan_to_value_ratio.isnull().sum()

99789

In [309]:
df.combined_loan_to_value_ratio.value_counts()

combined_loan_to_value_ratio
80.000    2165
95.000    1504
75.000    1189
96.500     986
90.000     860
          ... 
64.921       1
84.194       1
90.466       1
41.040       1
51.688       1
Name: count, Length: 10354, dtype: int64